In [1]:
from keras.utils import image_dataset_from_directory, set_random_seed
from keras.models import load_model
from collections import Counter

In [2]:
set_random_seed(10)

In [3]:
img_size = (256, 256)
class_names = ['NORMAL', 'PNEUMONIA']
dataset_dir = '/Users/snoopy/Desktop/Other/Harvard/Final Project/pediatric-chest-xrays/chest_xray'
color_mode = 'rgb'
num_channels = 3

In [4]:
train_data = image_dataset_from_directory(directory=dataset_dir+'/train',
                                          label_mode='binary',
                                          class_names=class_names,
                                          color_mode=color_mode,
                                          image_size=img_size,
                                          shuffle=True)
test_data = image_dataset_from_directory(directory=dataset_dir+'/test',
                                          label_mode='binary',
                                          class_names=class_names,
                                          color_mode=color_mode,
                                          image_size=img_size)
val_data = image_dataset_from_directory(directory=dataset_dir+'/val',
                                          label_mode='binary',
                                          class_names=class_names,
                                          color_mode=color_mode,
                                          image_size=img_size)

Found 5216 files belonging to 2 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


2022-08-02 08:48:26.241825: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-02 08:48:26.241938: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
train_len = train_data.__len__().numpy()
val_len = int(train_len * 0.2)
val_data = val_data.concatenate(train_data.take(val_len))
train_data = train_data.skip(val_len)

In [6]:
model = load_model(filepath='/Users/snoopy/Desktop/Other/Harvard/Final Project/RSNA/RSNA-full-resnet-augmented/rsna-resnet-augmented')

In [7]:
model.summary()

Model: "resnet_18_classifier"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 256, 256, 3)  0           ['input_1[0][0]']                
                                                                                                  
 bn_data (BatchNormalization)   (None, 256, 256, 3)  9           ['rescaling[0][0]']              
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0          ['bn_data[0][0

In [8]:
threshold = 0.5

In [9]:
train_res = model.evaluate(train_data, verbose=1)
val_res = model.evaluate(val_data, verbose=1)
test_res = model.evaluate(test_data, verbose=1)

2022-08-02 08:50:12.489985: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-02 08:50:12.490461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 3s 143ms/step - loss: 0.7274 - accuracy: 0.5705 - precision: 0.9919 - recall: 0.3154 - auc: 0.8760


In [10]:
train_pred = model.predict(train_data, verbose=1)
val_pred = model.predict(val_data, verbose=1)
test_pred = model.predict(test_data, verbose=1)

2022-08-02 08:50:43.143569: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 3s 140ms/step


In [11]:
train_pred_labels = []
for pred in train_pred:
    if pred>threshold:
        train_pred_labels.append(1)
    else:
        train_pred_labels.append(0)

val_pred_labels = []
for pred in val_pred:
    if pred>threshold:
        val_pred_labels.append(1)
    else:
        val_pred_labels.append(0)

test_pred_labels = []
for pred in test_pred:
    if pred>threshold:
        test_pred_labels.append(1)
    else:
        test_pred_labels.append(0)

print("Training predictions")
print(Counter(train_pred_labels))
print("\nValidation predictions")
print(Counter(val_pred_labels))
print('\nTest predictions')
print(Counter(test_pred_labels))

Training predictions
Counter({0: 3226, 1: 966})

Validation predictions
Counter({0: 797, 1: 243})

Test predictions
Counter({0: 500, 1: 124})


In [12]:
train_f1_from_eval = 2./(1./train_res[2]+1./train_res[3])
val_f1_from_eval = 2./(1./val_res[2]+1./val_res[3])
test_f1 = 2./(1./test_res[2]+1./test_res[3])

In [13]:
print('Train precision: ', train_res[2])
print("Train recall: ", train_res[3])
print("Train F1: ", train_f1_from_eval)
print("\nValidation precision: ", val_res[2])
print("Validation recall: ", val_res[3])
print("Validation F1: ", val_f1_from_eval)
print("\nTest precision: ", test_res[2])
print("Test recall: ", test_res[3])
print("Test F1: ", test_f1)

Train precision:  0.988469660282135
Train recall:  0.3033129572868347
Train F1:  0.4641890234019653

Validation precision:  0.9878049492835999
Validation recall:  0.31640625
Validation F1:  0.4792899492136638

Test precision:  0.9919354319572449
Test recall:  0.3153846263885498
Test F1:  0.4785992284173336
